In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [97]:
try:
    import community as community_louvain
    HAVE_LOUVAIN = True
except Exception:
    HAVE_LOUVAIN = False

In [2]:
ie = pd.read_csv('../../cfb_cs/ie_con_cc.csv')
ie_cont = pd.read_csv('../data/ie_contr.csv')
dt = pd.read_csv('../data/donor_totals.csv')

In [6]:
ie[ie['POSITION'] == 'Support'].groupby(['SPENDER_NAME','CANDNAME'])['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False).head(15)

,SPENDER_NAME,CANDNAME,ALLOCATION
157,POLICE BENEVOLENT ASSOCIATION INDEPENDENT EXPE...,"Albanese, Sal F",376596.36
159,Rebuild NYC 2021,"Menin, Julie",191152.27
158,POLICE BENEVOLENT ASSOCIATION INDEPENDENT EXPE...,"Ariola, Joann",179038.44
140,NY4KIDS Inc.,"Rein, Briget",171000.00
169,"Voters of NYC, Inc.","Diaz, Darma V",125775.00
67,Hotel Workers for Stronger Communities,"Hudson, Crystal",113675.11
133,NY4KIDS Inc.,"Dinowitz, Eric",105581.00
132,NY4KIDS Inc.,"Bravo, Ischia",98181.00
72,Hotel Workers for Stronger Communities,"Shafran, Austin",97026.66
37,"Common Sense NYC, Inc.","Schulman, Lynn",92426.00


In [3]:
ie.groupby('COMM_PURPOSE')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False)

,COMM_PURPOSE,ALLOCATION
5,Mass Mailing,3405408.64
3,Internet - Display/Banner,1347936.35
10,Video (Internet),538307.00
7,Phone Calls - Live,474041.02
11,Video (TV/Cable),268558.00
4,Leaflet/Flyer,198708.45
2,Billboard,100600.00
6,Other,96074.00
9,Robo Call,65447.02
8,Print Ad,63692.15


In [68]:
opp_list = ie[ie['POSITION'] == 'Oppose'].groupby(['SPENDER_NAME','CANDNAME'])['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False)['CANDNAME'].to_list()

In [21]:
prog_list = ie[(ie['CANDNAME'].isin(opp_list)) & (ie['POSITION'] == 'Support')].groupby('SPENDER_NAME')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False)['SPENDER_NAME'].to_list()

In [3]:
ie[ie['POSITION'] == 'Support'].groupby('CANDNAME')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False).head(15)

,CANDNAME,ALLOCATION
53,"Menin, Julie",387036.40
4,"Albanese, Sal F",386596.36
40,"Hudson, Crystal",240590.24
26,"De La Rosa, Carmen",237400.28
65,"Rein, Briget",209747.35
1,"Abreu, Shaun",207111.04
74,"Schulman, Lynn",190163.48
7,"Ariola, Joann",179038.44
27,"Diaz, Darma V",177972.95
75,"Shafran, Austin",157765.17


In [5]:
ie[ie['POSITION'] == 'Support'].groupby('CANDNAME')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False).describe()

,ALLOCATION
count,84.000000
mean,68385.808929
std,79550.157108
min,659.440000
25%,9553.287500
50%,38046.770000
75%,102048.327500
max,387036.400000


In [6]:
68385 + 79550

147935

In [3]:
ie.columns

Index(['ELECTION', 'SPENDERID', 'SPENDER_NAME', 'C_CODE', 'STRNO', 'STRNAME',
       'APARTMENT', 'CITY', 'STATE', 'ZIP', 'FILING', 'COMMUNICATION_ID',
       'COMMUNICATION_NAME', 'COMM_DATE', 'COMM_PURPOSECD', 'COMM_PURPOSE',
       'COMM_AMNT', 'CANDID', 'CANDNAME', 'OFFICECD', 'OFFICE_BORO_DISTRICT',
       'ALLOCATION', 'POSITION'],
      dtype='object')

In [12]:
ie[['SPENDER_NAME', 'C_CODE', 'FILING', 'COMMUNICATION_ID','COMMUNICATION_NAME','COMM_PURPOSE']].head()

,SPENDER_NAME,C_CODE,FILING,COMMUNICATION_ID,COMMUNICATION_NAME,COMM_PURPOSE
0,"A Better NY for All, LLC",LLC,5,3582,Creative and Paid Media Services,Other
1,"A Better NY for All, LLC",LLC,5,3582,Creative and Paid Media Services,Other
2,"A Better NY for All, LLC",LLC,5,3541,Digital Programmatic Ad Spend for Darma,Internet - Display/Banner
3,"A Better NY for All, LLC",LLC,9,3715,GOTV Election Day Operation,Other
4,"A Better NY for All, LLC",LLC,GD,3705,Magazine Ad,Print Ad


In [64]:
ie.groupby('SPENDER_NAME')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False)

,SPENDER_NAME,ALLOCATION
6,"Common Sense NYC, Inc.",1521828.45
13,Labor Strong 2021,1014211.03
15,NY4KIDS Inc.,897385.24
10,Hotel Workers for Stronger Communities,719841.33
20,POLICE BENEVOLENT ASSOCIATION INDEPENDENT EXPE...,633094.80
14,Laborers Building a Better New York,341784.47
23,"Voters of NYC, Inc.",313108.25
21,Rebuild NYC 2021,191152.27
8,Empire State 32BJ SEIU PAC,181848.85
0,"A Better NY for All, LLC",128475.28


In [16]:
ie.groupby('COMM_PURPOSE')['ALLOCATION'].sum().reset_index().sort_values('ALLOCATION',ascending=False)

,COMM_PURPOSE,ALLOCATION
5,Mass Mailing,3405408.64
3,Internet - Display/Banner,1347936.35
10,Video (Internet),538307.00
7,Phone Calls - Live,474041.02
11,Video (TV/Cable),268558.00
4,Leaflet/Flyer,198708.45
2,Billboard,100600.00
6,Other,96074.00
9,Robo Call,65447.02
8,Print Ad,63692.15


In [5]:
ie_cont.columns

Index(['ELECTION', 'OFFICECD', 'RECIPID', 'CANCLASS', 'RECIPNAME', 'COMMITTEE',
       'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO', 'REFNO', 'DATE',
       'REFUNDDATE', 'NAME', 'C_CODE', 'STRNO', 'STRNAME', 'APARTMENT',
       'BOROUGHCD', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME',
       'EMPSTRNO', 'EMPSTRNAME', 'EMPCITY', 'EMPSTATE', 'AMNT', 'MATCHAMNT',
       'PREVAMNT', 'PAY_METHOD', 'INTERMNO', 'INTERMNAME', 'INTSTRNO',
       'INTSTRNM', 'INTAPTNO', 'INTCITY', 'INTST', 'INTZIP', 'INTEMPNAME',
       'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA',
       'PURPOSECD', 'EXEMPTCD', 'ADJTYPECD', 'RR_IND', 'SEG_IND',
       'INT_C_CODE'],
      dtype='object')

In [122]:
ie_cont[['NAME', 'C_CODE','OCCUPATION', 'EMPNAME','AMNT', 'MATCHAMNT' ,'RECIPNAME']].sample(5)

,NAME,C_CODE,OCCUPATION,EMPNAME,AMNT,MATCHAMNT,RECIPNAME
454,"Nislick, Steve",IND,Consultant,Self,75000.0,0.0,New York Deserves Better PAC
180,"Gagosian, Laurence",IND,Art Dealer,Gagosian,5000.0,0.0,Rebuild NYC 2021
101,"Cohen, Steven",IND,Majority Owner,New York Mets,500000.0,0.0,Comeback PAC
17,AFT Solidarity,EMPO,NaN,NaN,100000.0,0.0,NY4KIDS Inc.
39,"Barad, Gerald",IND,Hospitality,Self,5000.0,0.0,Strong Leadership NYC Inc


In [17]:
ie_cont.groupby('C_CODE')['AMNT'].sum().reset_index().sort_values('AMNT',ascending=False)

,C_CODE,AMNT
2,IND,30950068.40
1,EMPO,11631725.63
4,OTHR,4432077.25
0,CORP,4426267.60
3,LLC,1044104.20
5,PART,100000.00


In [37]:
groups = {
    'Executive': ['CEO', 'Chief Executive Officer', 'Executive', 'Consultant/CEO','Executive Chairman'],
    'Investor': ['Investor', 'INVESTOR', 'Portfolio Investor'],
    'Retired/Unemployed': ['Retired', 'Not Employed']
}


In [38]:
mapping = {occ: group for group, occs in groups.items() for occ in occs}

In [39]:
ie_cont['grouped_occupation'] = ie_cont['OCCUPATION'].apply(
    lambda x: mapping.get(x, x)
)

In [32]:
ie_cont[(ie_cont['grouped_occupation'] == 'Executive') & ie_cont['RECIPNAME'].isin(prog_list)][['NAME','RECIPNAME','OCCUPATION','EMPNAME']]

,NAME,RECIPNAME,OCCUPATION,EMPNAME
234,"Hirschhorn, Michael",Our City,CEO,MEBL Transforming Furniture
570,"Souweine, Daniel",Our City,CEO,"Toskr, Inc."
666,"Williams, Sarah",Our City,CEO,Propel


In [59]:
groups = {
    'Executive': ['CEO', 'Chief Executive Officer', 'Executive', 'Consultant/CEO','Executive Chairman'],
    'Investor': ['Investor', 'INVESTOR', 'Portfolio Investor'],
    'Retired/Unemployed': ['Retired', 'Not Employed','Not employed'],
    'Attorney' : ['Attorney','Lawyer']
}


In [60]:
mapping = {occ: group for group, occs in groups.items() for occ in occs}

In [61]:
ie_cont['grouped_occupation'] = ie_cont['OCCUPATION'].apply(
    lambda x: mapping.get(x, x)
)

In [31]:
ie_cont[(ie_cont['C_CODE'] == 'IND') & ie_cont['RECIPNAME'].isin(prog_list)].groupby('grouped_occupation')['AMNT'].sum().reset_index().sort_values('AMNT',ascending=False).head(10)

,grouped_occupation,AMNT
12,Executive Director,224000.0
23,Retired/Unemployed,65500.0
17,Organizer,11000.0
4,Attorney,9100.0
2,Art Critic,8500.0
20,Professor,8000.0
22,Publisher,5000.0
11,Executive,4500.0
9,Donor organizer,2500.0
10,Educator,2000.0


In [ ]:
ie_cont[ie_cont['grouped_occupation'] == 'Executive'].groupby('grouped_occupation')['AMNT'].sum().reset_index().sort_values('AMNT',ascending=False).head(10)

In [48]:
ie[['SPENDER_NAME','CANDNAME', 'ALLOCATION','POSITION']].shape

(752, 4)

In [47]:
ie[['SPENDER_NAME','CANDNAME', 'ALLOCATION','POSITION']].sample(5)

,SPENDER_NAME,CANDNAME,ALLOCATION,POSITION
347,Hotel Workers for Stronger Communities,"Sanchez, Pierina A",14488.69,Support
514,Labor Strong 2021,"De La Rosa, Carmen",1952.00,Support
28,Amplify Her PAC,"Velazquez, Marjorie",166.67,Support
326,Hotel Workers for Stronger Communities,"Hudson, Crystal",25329.74,Support
231,"Common Sense NYC, Inc.","Choe, John",14237.50,Oppose


In [78]:
ie['agg_all'] = np.where(ie['POSITION'] == 'Support', (ie['ALLOCATION'] * 1), (ie['ALLOCATION'] * -1))

In [53]:
dt.sample()

,NAME,CANDID,indirect_support_total,indirect_opp_total,indirect_total,CANDNAME
8799,"Hirschhorn, Michael",2207,0.0,0.0,0.0,"Louis, Farah"


In [62]:
dt_occ = dt.merge(ie_cont[['NAME','grouped_occupation','OCCUPATION','EMPNAME']],on='NAME')

In [65]:
dt_occ[dt_occ['CANDNAME'].isin(opp_list)].groupby(['NAME','grouped_occupation','EMPNAME'])['indirect_opp_total'].sum().reset_index().sort_values('indirect_opp_total',ascending=False).head(10)

,NAME,grouped_occupation,EMPNAME,indirect_opp_total
102,"Ross, Stephen M",Investor,SELF EMPLOYED,1000000.0
67,"LAUDER, RONALD",Investor,SELF EMPLOYED,500000.0
14,"Cayre, Jack",Investor,SELF EMPLOYED,214000.0
71,"Lauder, Ronald S",Philanthropist,Lauder Foundation,100000.0
3,"Ash, Isaac",Executive,UNITED LEGWEAR & APPAREL CO.,100000.0
123,"Tamir, Elliot",Co Founder,Vesper Holdings,100000.0
106,"SCHARF, RICHARD",President,Abro Management Corp,100000.0
131,"Weinstein, Elliot",Director,TL Management,50000.0
15,"Chetrit, Joseph",Principal,Chetrit Group,50000.0
42,"Goldman, Lloyd",founder and president,BLDG Management,25000.0


In [66]:
dt_occ[dt_occ['CANDNAME'].isin(opp_list)].groupby(['NAME','grouped_occupation','EMPNAME'])['indirect_support_total'].sum().reset_index().sort_values('indirect_support_total',ascending=False).head(10)

,NAME,grouped_occupation,EMPNAME,indirect_support_total
119,"Speranza, Marti",Executive Director,Amplify Her,332946.419410
78,"Linker, Kate",Art Critic,Self-employed,19125.000000
39,"Galowitz, Paula",Retired/Unemployed,NYU Law School,15000.000000
4,"Baldwin, Peter",Professor,UCLA,12000.000000
63,"Kattan, Christine",Family Manager,Not Applicable,11928.340641
5,"Barkan, Ady",Organizer,Center for Popular Democracy,7500.000000
124,"Tobias, Gabriel",Executive Director,Our City PAC,6000.000000
130,"Vanden Heuvel, Katrina",Publisher,The Nation,3750.000000
56,"Holme Elsberg, Natalie",Retired/Unemployed,Not employed,3000.000000
125,"Tobias, Gabriel",Organizer,Not Applicable,3000.000000


In [47]:
ie_cont[ie_cont['NAME'].isin(person_list)][['NAME','OCCUPATION','EMPNAME']]

,NAME,OCCUPATION,EMPNAME
4,32BJ United American Dream Fund,NaN,NaN
12,AFSCME,NaN,NaN
18,AFT Solidarity 527,NaN,NaN
34,"Baldwin, Peter",Professor,UCLA
35,"Baldwin, Peter",Professor,UCLA
40,"Barkan, Ady",Organizer,Center for Popular Democracy
108,Communications Workers of America District One...,NaN,NaN
109,Communications Workers of America District One...,NaN,NaN
115,"Costello, Eric",Not employed,NaN
120,CWA District One PAC,NaN,NaN
